In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import warnings

import tensorflow as tf
import keras as ke
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers import AveragePooling1D
from keras.losses import categorical_crossentropy
from keras.optimizers import RMSprop
from keras.models import Sequential
from tflearn.data_utils import to_categorical

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import pickle 
import json
import requests

warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'tensorflow'

In [65]:
from google.colab import drive
drive.mount('/content/gdrive')

data = arff.loadarff('/content/gdrive/My Drive/Training Dataset.arff')
data = pd.DataFrame(data[0])

data.head(2)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,b'-1',b'1',b'1',b'1',b'-1',b'-1',b'-1',b'-1',b'-1',b'1',b'1',b'-1',b'1',b'-1',b'1',b'-1',b'-1',b'-1',b'0',b'1',b'1',b'1',b'1',b'-1',b'-1',b'-1',b'-1',b'1',b'1',b'-1',b'-1'
1,b'1',b'1',b'1',b'1',b'1',b'-1',b'0',b'1',b'-1',b'1',b'1',b'-1',b'1',b'0',b'-1',b'-1',b'1',b'1',b'0',b'1',b'1',b'1',b'1',b'-1',b'-1',b'0',b'-1',b'1',b'1',b'1',b'-1'


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 31 columns):
having_IP_Address              11055 non-null object
URL_Length                     11055 non-null object
Shortining_Service             11055 non-null object
having_At_Symbol               11055 non-null object
double_slash_redirecting       11055 non-null object
Prefix_Suffix                  11055 non-null object
having_Sub_Domain              11055 non-null object
SSLfinal_State                 11055 non-null object
Domain_registeration_length    11055 non-null object
Favicon                        11055 non-null object
port                           11055 non-null object
HTTPS_token                    11055 non-null object
Request_URL                    11055 non-null object
URL_of_Anchor                  11055 non-null object
Links_in_tags                  11055 non-null object
SFH                            11055 non-null object
Submitting_to_email            11055 non-nu

In [67]:
categorical_feature_mask = data.dtypes==object
categorical_cols = data.columns[categorical_feature_mask].tolist()
le = LabelEncoder()
data[categorical_cols] = data[categorical_cols].apply(lambda col: le.fit_transform(col))
data[categorical_cols].head(2)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,0,2,1,1,0,0,0,0,0,1,1,0,1,0,2,0,0,0,0,1,1,1,1,0,0,0,0,1,2,0,0
1,1,2,1,1,1,0,1,2,0,1,1,0,1,1,0,0,1,1,0,1,1,1,1,0,0,1,0,1,2,1,0


In [ ]:
x = data.iloc[ : , :-1].values
y = data.iloc[:, -1:].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler((-1,1))
x = sc.fit_transform(x)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.1)

In [71]:
treemod = DecisionTreeClassifier(criterion = 'gini',min_samples_leaf = 10 , min_impurity_decrease = 0.01)
treemod.fit(xtrain,ytrain)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.01, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [72]:
pred = treemod.predict(xtest)
cm = confusion_matrix(ytest,pred)
cm

array([[436,  40],
       [ 47, 583]])

In [73]:
correct_pred = sum([cm[k][k] for k in range(cm.shape[0])])
acc = correct_pred/xtest.shape[0]
print('Decision Tree Accuracy: {}'.format(acc))

Decision Tree Accuracy: 0.9213381555153707


In [ ]:
pred = treemod.predict(x)
data['DT'] = pred

In [75]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg')
log.fit(xtrain, ytrain)
pred = log.predict(xtest)
cm = confusion_matrix(ytest,pred)
correct_pred = sum([cm[k][k] for k in range(cm.shape[0])])
acc = correct_pred/xtest.shape[0]
print('Linear Model Accuracy: {}'.format(acc))

Linear Model Accuracy: 0.9358047016274864


In [ ]:
pred = log.predict(x)
data['LR'] = pred

In [77]:
svc = SVC(kernel = 'rbf')
svc.fit(xtrain,ytrain)
pred = svc.predict(xtest)
cm = confusion_matrix(ytest,pred)
correct_pred = sum([cm[k][k] for k in range(cm.shape[0])])
acc = correct_pred/xtest.shape[0]
print('SVM Rbf Accuracy: {}'.format(acc))

SVM Rbf Accuracy: 0.9511754068716094


In [ ]:
pred = svc.predict(x)
data['SVM'] = pred

In [ ]:
r_state = random.randint(0,round(random.random() * 1000))

In [80]:
max_i = 0
max_acc = 0
for i in [j*10 for j in range(1,10)]:
 rf = RandomForestClassifier(max_depth = i, random_state=r_state)
 rf.fit(xtrain, ytrain)
 pred = rf.predict(xtest)
 cm = confusion_matrix(ytest,pred)
 correct_pred = sum([cm[k][k] for k in range(cm.shape[0])])
 acc = correct_pred/xtest.shape[0]
 print('Max Depth: {} \tAccuracy: {}'.format(i,acc))
 if(acc > max_acc):
  max_acc = acc
  max_i = i

Max Depth: 10 	Accuracy: 0.9620253164556962
Max Depth: 20 	Accuracy: 0.9755877034358047
Max Depth: 30 	Accuracy: 0.9773960216998192
Max Depth: 40 	Accuracy: 0.9773960216998192
Max Depth: 50 	Accuracy: 0.9773960216998192
Max Depth: 60 	Accuracy: 0.9773960216998192
Max Depth: 70 	Accuracy: 0.9773960216998192
Max Depth: 80 	Accuracy: 0.9773960216998192
Max Depth: 90 	Accuracy: 0.9773960216998192


In [81]:
max_dep = 0
max_acc = 0
for i in [j for j in range(max_i-10,max_i+10)]:
 rf = RandomForestClassifier(max_depth = i, random_state=r_state)
 rf.fit(xtrain , list(ytrain))
 pred = rf.predict(xtest)
 cm = confusion_matrix(ytest,pred)
 correct_pred = sum([cm[k][k] for k in range(cm.shape[0])])
 acc = correct_pred/xtest.shape[0]
 print('Max Depth: {} \tAccuracy: {}'.format(i,acc))
 if(acc > max_acc):
  max_dep = i
  max_acc = acc
print('Best Max Depth: {}'.format(max_dep))

Max Depth: 20 	Accuracy: 0.9755877034358047
Max Depth: 21 	Accuracy: 0.9792043399638336
Max Depth: 22 	Accuracy: 0.976491862567812
Max Depth: 23 	Accuracy: 0.9755877034358047
Max Depth: 24 	Accuracy: 0.9773960216998192
Max Depth: 25 	Accuracy: 0.9746835443037974
Max Depth: 26 	Accuracy: 0.9773960216998192
Max Depth: 27 	Accuracy: 0.9773960216998192
Max Depth: 28 	Accuracy: 0.9773960216998192
Max Depth: 29 	Accuracy: 0.9773960216998192
Max Depth: 30 	Accuracy: 0.9773960216998192
Max Depth: 31 	Accuracy: 0.9773960216998192
Max Depth: 32 	Accuracy: 0.9773960216998192
Max Depth: 33 	Accuracy: 0.9773960216998192
Max Depth: 34 	Accuracy: 0.9773960216998192
Max Depth: 35 	Accuracy: 0.9773960216998192
Max Depth: 36 	Accuracy: 0.9773960216998192
Max Depth: 37 	Accuracy: 0.9773960216998192
Max Depth: 38 	Accuracy: 0.9773960216998192
Max Depth: 39 	Accuracy: 0.9773960216998192
Best Max Depth: 21


In [ ]:
pred = rf.predict(x)
data['RF'] = pred

In [83]:
data.head(2)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result,DT,LR,SVM,RF
0,0,2,1,1,0,0,0,0,0,1,1,0,1,0,2,0,0,0,0,1,1,1,1,0,0,0,0,1,2,0,0,0,0,0,0
1,1,2,1,1,1,0,1,2,0,1,1,0,1,1,0,0,1,1,0,1,1,1,1,0,0,1,0,1,2,1,0,1,1,0,0


In [84]:
xgbmod = XGBClassifier(learning_rate =0.3,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgbmod.fit(xtrain,ytrain)
pred = xgbmod.predict(xtest)
cm = confusion_matrix(ytest,pred)
cm

array([[460,  16],
       [ 12, 618]])

In [85]:
correct_pred = sum([cm[k][k] for k in range(cm.shape[0])])
acc = correct_pred/xtest.shape[0]
print('XGBoost Accuracy: {}'.format(acc))

XGBoost Accuracy: 0.9746835443037974


In [ ]:
pred = xgbmod.predict(x)
data['XGB'] = pred

In [87]:
data.head(2)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result,DT,LR,SVM,RF,XGB
0,0,2,1,1,0,0,0,0,0,1,1,0,1,0,2,0,0,0,0,1,1,1,1,0,0,0,0,1,2,0,0,0,0,0,0,0
1,1,2,1,1,1,0,1,2,0,1,1,0,1,1,0,0,1,1,0,1,1,1,1,0,0,1,0,1,2,1,0,1,1,0,0,0


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 36 columns):
having_IP_Address              11055 non-null int64
URL_Length                     11055 non-null int64
Shortining_Service             11055 non-null int64
having_At_Symbol               11055 non-null int64
double_slash_redirecting       11055 non-null int64
Prefix_Suffix                  11055 non-null int64
having_Sub_Domain              11055 non-null int64
SSLfinal_State                 11055 non-null int64
Domain_registeration_length    11055 non-null int64
Favicon                        11055 non-null int64
port                           11055 non-null int64
HTTPS_token                    11055 non-null int64
Request_URL                    11055 non-null int64
URL_of_Anchor                  11055 non-null int64
Links_in_tags                  11055 non-null int64
SFH                            11055 non-null int64
Submitting_to_email            11055 non-null int64
Abnorma

In [2]:
data.corr()

NameError: name 'data' is not defined

In [3]:
x = data.loc[:,['SVM','RF','XGB']].values
y = data.loc[:,['Result']].values

NameError: name 'data' is not defined

In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2)

NameError: name 'train_test_split' is not defined